# Analysis of intermetallic precipitates in Al matrix

In this notebook, we will focus on a small region of the field of view for a closer analysis of the precipitates and grain boundaries.

Make sure to have run the [keypoint-registration](./keypoint-registration.ipynb) notebook first and have saved the warped EBSD and misorientation arrays!

```{admonition} Acknowledgements
This notebook was adapted from a [project](https://gitlab.com/epfl-center-for-imaging/ebsd-precipitates) that was part of a collaboration between the EPFL Center for Imaging and the Thermomechanical Metallurgy Laboratory.
```

In [ ]:
from pathlib import Path
import napari
import tifffile
import numpy as np
import pandas as pd
from scipy.ndimage import label, distance_transform_edt
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import remove_small_objects, skeletonize
from skimage.filters import gaussian
from skimage.feature import peak_local_max
from skimage.segmentation import watershed, find_boundaries
from skimage.exposure import rescale_intensity

# Imports from this project's packages:
import sys; sys.path.append('..')

from gefolki.python.algorithm import EFolki
from gefolki.python.tools import wrapData

We keep all our data in this folder.

In [ ]:
data_folder = Path('..') / 'data'

Load the data.

In [ ]:
sem = tifffile.imread(data_folder / 'sem.tif')

# Outputs from the previous notebook (keypoint-registration.ipynb)
warped_ebsd_rgb = tifffile.imread(data_folder / 'warped_ebsd_rgb.tif')
warped_misorientation = tifffile.imread(data_folder / 'warped_misorientation.tif')

# This is the output of the Ilastik model we used to segment the precipitates.
ilastik_precipitates_segmentation = np.load(data_folder / 'simple_segmentation.npy')

For simplicity and speed, we select a region of interest in the field of view by cropping all the arrays.

In [ ]:
start_x = 4000
end_x = 6000
start_y = 4000
end_y = 6000

sem = sem[start_x:end_x, start_y:end_y]
warped_ebsd_rgb = warped_ebsd_rgb[start_x:end_x, start_y:end_y]
warped_misorientation = warped_misorientation[start_x:end_x, start_y:end_y]
ilastik_precipitates_segmentation = ilastik_precipitates_segmentation[start_x:end_x, start_y:end_y]

We identify grain boundaries based on a pre-defined angle threshold (e.g. 15 deg.).

In [ ]:
grain_boundary_angle_threshold_deg = 15.0
grain_boundaries = warped_misorientation >= grain_boundary_angle_threshold_deg

Visualize the arrays in Napari.

In [ ]:
viewer = napari.Viewer()
viewer.add_image(sem, name='Backscatter SEM')
viewer.add_image(warped_misorientation, colormap='inferno', opacity=0.7, name='Misorientation (deg.)')
viewer.add_labels(grain_boundaries)

Next, we aim at producing a grain segmentation from our grain boundary array. For this, we use a watershed transform. The watershed algorithm also requires that we find peaks in the distance map of the grain boundaries to be used as markers. Ideally, there should be one marker per grain.

We use the boundaries of the watershed segmentation as our new reference for the grain boundary map - which is a closed network of grain boundaries.

In [ ]:
# Compute the distance transform
distance_img = distance_transform_edt(grain_boundaries == 0)

# Find peaks
peaks_data = peak_local_max(
    -distance_img, 
    min_distance=80,
    threshold_abs=None,
    threshold_rel=None, 
    exclude_border=False
)

# Extract markers
markers = np.zeros_like(distance_img)
markers[tuple(peaks_data.T)] = 1
markers, _ = label(markers)

# Watershed segmentation
watershed_grain_segmentation = watershed(-distance_img, markers)

watershed_grain_boundaries = find_boundaries(watershed_grain_segmentation, connectivity=1, mode='thick')

# We keep only the biggest connected object to remove unwanted artifacts inside the grains.
labels = label(watershed_grain_boundaries)[0]  # label from scipy
counts = np.unique(labels, return_counts=1)
biggestLabel = np.argmax(counts[1][1:]) + 1
watershed_grain_boundaries = (labels == biggestLabel).astype(int)

Look at the results.

In [ ]:
# Display the distance map (for info)
viewer.add_image(distance_img, colormap='viridis', name="Distance map")

# Display the segmentation in a `Labels` layer
viewer.add_labels(watershed_grain_segmentation, name='Watershed grain segmentation')

# Display the new grain boundary map
viewer.add_labels(watershed_grain_boundaries, name="Watershed grain boundaries")

## Fine-tuning the EBSD-SEM registration



In [ ]:
ebsd_mean_normalized = rescale_intensity(np.mean(warped_ebsd_rgb, axis=2), out_range=(0, 1))
sem_normalized = rescale_intensity(sem, out_range=(0, 1))

u, v = EFolki(
    sem_normalized, 
    ebsd_mean_normalized, 
    iteration=2, 
    radius=[32, 24, 16, 8], 
    rank=4, 
    levels=5
)

N = np.sqrt(u**2+v**2)

Next, we correct our grain boundary map by applying the fine-tuned registration transform to it. We then recompute the distance transform to be able to tell how far the precipitates are from the grain boundaries.

In [ ]:
blurred_grain_boundaries = gaussian(watershed_grain_boundaries, sigma=5)
blurred_grain_boundaries = rescale_intensity(blurred_grain_boundaries, out_range=(0, 1))

warped_blurred_grain_boundaries = wrapData(blurred_grain_boundaries, u, v)
warped_blurred_grain_boundaries = rescale_intensity(warped_blurred_grain_boundaries, out_range=(0, 1))

warped_blurred_grain_boundaries_skeleton = skeletonize(warped_blurred_grain_boundaries > 0.1)
distance_img_corrected = distance_transform_edt(warped_blurred_grain_boundaries_skeleton == 0)

Look at the results.

In [ ]:
viewer.add_image(warped_blurred_grain_boundaries, colormap='red', blending='additive', opacity=0.5, name="Warped grain boundaries")
viewer.add_labels(warped_blurred_grain_boundaries_skeleton, name="Boundaries skeleton")
viewer.add_image(distance_img_corrected, name="Distance map (corrected)", blending="additive", opacity=0.5, colormap='viridis')

## Precipitates

Next, we prepare our precipitates array. We remove precipitates below or above pre-determined sizes. We distinguish individual precipitates by applying connected components labeling to the array.

In [ ]:
# 1 is the precipitates, 0 the matrix, 2 the intermetallics..
labeled_precipitates, _ = label(np.squeeze((ilastik_precipitates_segmentation == 1).astype(np.uint8)))

# Remove small objects
min_size = 500

labeled_precipitates = remove_small_objects(labeled_precipitates, min_size=min_size)
labeled_precipitates = label(labeled_precipitates)[0]

print(f'Precipitates with size above {min_size}: {len(np.unique(labeled_precipitates))}')

# Remove large objects
max_size = 2000

props = regionprops(labeled_precipitates)
large_objects = [p.label for p in props if p.area > max_size]
labeled_precipitates[np.isin(labeled_precipitates, large_objects)] = 0

Look at the results.

In [ ]:
viewer.add_labels(labeled_precipitates, name="Precipitates segmentation")

Then, we extract the centroid (location) of each precipitate and keep this information in a Pandas dataframe. We also recall the distance between each centroid and the closest grain boundary by looking up the distance map.

In [ ]:
df = pd.DataFrame(data=regionprops_table(labeled_precipitates, properties=['centroid', 'label']))
df['centroid-0'] = df['centroid-0'].astype(int)
df['centroid-1'] = df['centroid-1'].astype(int)

centroids = []
distances = []
for k, row in df[['centroid-0', 'centroid-1']].iterrows():
    x = row['centroid-0']
    y = row['centroid-1']
    d = distance_img_corrected[x, y]
    distances.append(d)
    centroids.append([x, y])

We determine a minimum distance in pixels from the closest boundary, from which the precipitates are considered to be in the bulk of the grain.

In [ ]:
pixel_threshold = 20 # distance from the grain boundaries, in pixels, to consider

# Grain centroids falling in this mask will be considered "on grain"
on_grain_mask = (distance_img_corrected < pixel_threshold).astype(int)

dists = []
on_grains = []
composite_image = np.zeros_like(distance_img_corrected)
for _, (x, y, lab) in df[['centroid-0', 'centroid-1', 'label']].iterrows():
    filt = labeled_precipitates == lab
    dist = distance_img_corrected[x, y]
    on_grain = dist <= pixel_threshold
    dists.append(dist)
    on_grains.append(on_grain)
    composite_image[filt] = dist

We generate one final visualization with textual information in a Points layer.

In [ ]:
l = viewer.add_labels(on_grain_mask, name="On grain mask")
l.contour = 3

viewer.add_image(composite_image, contrast_limits=(0, 100), colormap='inferno', opacity=0.5)

# Add that distance in text
points_props = {
    'name': 'distance (px)', 
    'face_color': 'orange', 
    'edge_width': 0, 
    'size': 20,
    'features': {'distance': dists, 'on_grains': on_grains}, 
    'text': {
        'string': 'Distance: {distance:0.0f} px\nOn grain: {on_grains}',
        'size': 9,
        'color': 'white'
    }
}

viewer.add_points(centroids, **points_props)